# Introduction

## About Quantum Computing Service

*Run quantum computing programs on Google’s quantum processors*

Quantum Computing Service gives customers access to Google's quantum computing hardware.  Programs that are written in Cirq, an open-source quantum computing program language, can be sent to run on a quantum computer in Google’s quantum computing lab in Santa Barbara, CA. You will be able to do  arbitrary single qubit rotations as well as several choices of two qubit gates.  Measurements of all qubits as a final step are also supported. 

Access is currently only granted to those on an approved list.  Apply to become an early access partner with this [questionnaire](https://docs.google.com/forms/d/1DfUWu4zUAJ87GKy-ZoTHrFri5IwIteKtMxKfsy3lmHE).


## Concepts

Quantum Computing Service uses Cirq, an open source Python framework for creating quantum programs and running them against simulators or against real quantum computers.    Here we provide a conceptual overview of how to run a quantum program on a quantum processor.

Installing cirq:

In [ ]:
!pip install --quiet cirq

     |████████████████████████████████| 1.4MB 5.3MB/s 


### Circuits

The language of quantum computing is the quantum circuit model.  Cirq is our open source framework which allows one to write a quantum circuit model in Python.  To learn more about Cirq itself, we recommend starting with the [Tutorial](../tutorials/basics.ipynb).  Conceptually Cirq can be thought of as a way to create an object in Python that contains the information about a quantum circuit.  Here, for example, we create a single circuit made up of square root of NOT gates, controlled-Z gates, and a measurement:

In [ ]:
import cirq

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    (cirq.X ** 0.5).on_each(q0, q1, q2),
    cirq.CZ(q0, q1),
    cirq.CZ(q1, q2),
    cirq.measure(q2, key='m')
)
print(circuit)

0: ───X^0.5───@────────────────
              │
1: ───X^0.5───@───@────────────
                  │
2: ───X^0.5───────@───M('m')───


### Quantum Engine API

Quantum Engine is the name of the cloud API which one can call to run the circuits you create in Cirq on simulator or on quantum computers.  When access is enabled, users can write code in Cirq that makes a call to Google’s servers to run a circuit that they have built in Cirq on simulators or on quantum computers.  Read more about how to do this using cirq's `Engine` class [here](./engine.ipynb).

![Quantum Engine Conceptual Diagram](../images/engine_diagram.png)

### Quantum Programs

In Cirq, one creates a `Circuit` in Python.  If one then wants to run it using Quantum Engine, one must then upload this Circuit (or Schedule) to the Quantum Engine.  The uploaded version of the Circuit/Schedule is called a Program.  Programs are not the Python code itself, but a representation of the Circuit in a hardware specific language.  Though you will need to make sure that your circuit uses only gates and qubits compatible with the hardware (see `cirq.google.optimized_for_sycamore()` to help with this), the `Engine` class and its corresponding `sampler` will translate the circuit to this serialized form for you.

![Quantum Program Conceptual Diagram](../images/engine_program.png)

When you upload a program to Quantum Engine, the program is given a name.  The organizational unit of Quantum Engine is the cloud project (see below), so when you create a program it is identified by the project id and the program id.  We write this as a path

```
<PROJECT_ID>/programs/<PROGRAM_ID>
```


### Cloud Project

Quantum Engine interfaces with Google Cloud.  In Google Cloud one can create different projects with different names.  For instance, you might create a project that is for your experiments on quantum computers, and you might create one to manage your side business running servers that day trade Star Wars memorabilia.  Everything under your cloud project can be seen by navigating to the Google Cloud console.  Here, for example, we see the programs that have been uploaded to a quantum projected called “Quantum Cloud Client”:

![Quantum Cloud Conceptual Diagram](../images/engine_cloud.png)

See the [Getting Started](../tutorials/google/start.ipynb) guide for step-by-step instructions on how to create a project and enable the API.

### Jobs

Once a circuit has been uploaded as a program to Quantum Engine, you can run this program by creating a job.  When you create a job on Quantum Engine, the quantum engine then is responsible for routing your program and the information about the job to the appropriate simulator or quantum computer.  Note that there is a time limit for job length, so you will want to separate your programs into multiple jobs below this limit.

![Quantum Jobs Conceptual Diagram](../images/engine_job.png)

Quantum Engine is designed so that you can upload a program once and then, if that program has different parameters (see below), you can run this program multiple times by creating multiple jobs for a given program.  Jobs are associated with programs, so they have names which include everything about your program, plus a job id:

```
<project id>/programs/<program id>/jobs/<job id>  
```

### Parameterized Circuits and Job Context

Circuits in Cirq can have quantum gates that are “parameterized”.  For example, a gate may represent a rotation of a qubit about the Z axis of the bloch sphere by an angle theta.  For parameterized circuit this angle theta is just specified by a variable, say “my-theta”.  In order to run such a circuit you will need to be able to tell the circuit what value of “my-theta” should be supplied.  In fact you can create a job for a program with parameterized gates which says run this for multiple parameter values.  Or each time you create a new job, you can specify new parameter values.  These parameter values are supplied in a “job context".

### Results

Once a job has been scheduled to run on a machine (simulator or quantum computer), the machine returns the results of the program to the Quantum Engine.  The Quantum Engine stores these results.  Users can then query for these results (typically one pulls every fraction of a second):

![Quantum Results Conceptual Diagram](../images/engine_result.png)

There is only one result for one job, so results can be identified via the identity of the job followed by result:

```
<project id>/programs/<program id>/jobs/<job id>/result
```

### Processors

When creating a job to run a program, one must specify where the job should run.  This can be a very specific machine to run on, or it can be specified more generally.  Each location that we can schedule a job to run on is described as a processor.  Different processors are available to different users, so processors appear under a project and have a processor id.

```
<project id>/processors/<processor id>
```

For example, one processor is the “rainbow” processor, which corresponds to a quantum computer located in Santa Barbara.  Another processor is “xmonsim” which is a fleet of simulators for the xmon architecture which all run in Google’s production datacenters.

Processor have state associated with them.  For instance, a processor can be “online” or they can be in “maintenance” mode.

### Calibration Metrics

Processors that are quantum computers periodically undergo calibrations to maintain the quality of the programs that can be run on these processors.  During this calibration metrics about the performance of the quantum computer is collected.  This calibration data is stored by Quantum Engine and users can then query for the state of the calibration (one can ask for the latest calibration, or what the state of calibration was at a given point in time).  Calibrations are also available for past jobs.

Calibration metrics and can be retrieved via the console or via cirq calls.  See more details on the [Calibration Metrics](calibration.ipynb) page.

### Reservations

Processor schedules are managed by a reservation system, assigning blocks of time as "time slots" with four possible states:

*  OPEN_SWIM: Anyone with processor access may run jobs.  There may be additional restrictions per processor that restrict large jobs.
*  MAINTENANCE: restrictions behave the same as during open swim, but there are no guarantees about the availability of the processor or quality of jobs run during this period.
*  RESERVATION: The processor is reserved for restricted use by a user or set of users. Reservations are rooted under a project and editors of that project may run jobs during that project's reservations while using any project they wish. Additional users may also be whitelisted to specific reservations.
*  UNALLOCATED: The processor has not been scheduled for any purpose. A reservation may be made during this time. If nothing is scheduled during this block, it will default to behaving as open swim.

During non-reservation times, jobs are restricted to run for at most 5 minutes and may have no more than 1,500,000 total shots on the processor.

The schedule of a processor is "frozen" for 4 hours into the immediate future so that users can rely on it for the coming day.

The ability to make reservations is restricted by budgets that are delegated by Quantum Computing Service admins. These are provided on a weekly basis and communicated either during the weekly user sync or by email and are good for one week in the future, but this policy is subject to change.

You can use this [reservation colab utility](../tutorials/google/reservations.ipynb)  to view the processor schedule and make reservations.

## Links to get started

[Getting started colab](../tutorials/google/start.ipynb)

Learn how to use the [Engine class](engine.md) in cirq.

Learn more about the [devices and gate sets](specification.md) available on the devices.

[Google Cloud Platform Console](https://console.cloud.google.com/apis/library/quantum.googleapis.com)

[Download a ipython template](../tutorials/google/colab.ipynb) to use in colaboratory

